In [1]:
import refexps
from os import listdir, path
from itertools import combinations
from collections import defaultdict
import numpy as np
from scipy import stats
from scipy.stats import mannwhitneyu
from refexps import get_previous, lex_material

In [8]:
### MOVE LEVEL ###
def compute_values_move(refexps_dir, success_dir, compound = True):
    move_alignment = defaultdict(list)
    for dialogue in listdir(refexps_dir):
        dialogue = path.join(refexps_dir, dialogue)
        dialogue = refexps.load_data(dialogue)
        #move level
        dialogue_move = refexps.move_level(dialogue)
        speakers = set()
        for move in dialogue_move:
            refs = move[1]
            for obj in refs:
                refobj = dialogue_game[obj]
                align_speaker = {}
                total_n = len(refs[obj])
                sameform_n = 0
                sameform_n_speaker = defaultdict(int)
                total_n_speaker = defaultdict(int)
                prev_forms = set()
                for ref in refs[obj]:
                    form = ref[3]
                    speaker = ref[1]
                    if prev_forms != set():
                        if form in prev_forms:
                            sameform_n += 1
                            sameform_n_speaker[speaker] += 1
                    else:
                        starter = speaker
                    speakers.add(speaker)
                    total_n_speaker[speaker] += 1
                    prev_forms.add(form)
                if (total_n - 1) != 0:
                    alignment = float(sameform_n)/float(total_n - 1)
                    move_alignment['Alignment'].append(alignment)
                for s in speakers:
                        if starter == s:
                            if (total_n_speaker[s] - 1) != 0:
                                alignment = float(sameform_n_speaker[s])/float(total_n_speaker[s] - 1)
                                move_alignment['Alignment '+s].append(alignment)
                        else:
                            if(total_n_speaker[s]) != 0:
                                alignment = float(sameform_n_speaker[s])/float(total_n_speaker[s])
                                move_alignment['Alignment '+s].append(alignment)
    for value in move_alignment:
        dataset = move_alignment[value]
        print '\t'+ value + ': '
        print '\tMean: '+ str(np.mean(dataset)) +'\tStandard deviation: '+ str(np.std(dataset))
        print '\tMax value: ' + str(max(dataset)) + '\tMin value: '+ str(min(dataset))
    return move_alignment

In [9]:
de_dataset_dir = 'En_De_Dataset/De/RefExp'
de_success_dir = 'En_De_Dataset/De/Success'

en_dataset_dir = 'En_De_Dataset/En/RefExp'
en_success_dir = 'En_De_Dataset/En/Success'

all_dataset_dir = 'En_De_Dataset/All/RefExp'
all_success_dir = 'En_De_Dataset/All/Success'

results_de = compute_values_move(de_dataset_dir, de_success_dir)
print '\n\n'
results_en = compute_values_move(en_dataset_dir, en_success_dir)
print '\n\n'
results_all = compute_values_move(all_dataset_dir, all_success_dir)

	Alignment p-utts: 
	Mean: 0.704540873908	Standard deviation: 0.349188315731
	Max value: 1.0	Min value: 0.0
	Alignment: 
	Mean: 0.70487088191	Standard deviation: 0.327073298495
	Max value: 1.0	Min value: 0.0
	Alignment e-utts: 
	Mean: 0.722099303247	Standard deviation: 0.360627429558
	Max value: 1.0	Min value: 0.0



	Alignment p-utts: 
	Mean: 0.71585324351	Standard deviation: 0.289371532332
	Max value: 1.0	Min value: 0.0
	Alignment: 
	Mean: 0.714459914457	Standard deviation: 0.282293848021
	Max value: 1.0	Min value: 0.0
	Alignment e-utts: 
	Mean: 0.752243395234	Standard deviation: 0.345331183265
	Max value: 1.0	Min value: 0.0



	Alignment p-utts: 
	Mean: 0.708120447251	Standard deviation: 0.331471884584
	Max value: 1.0	Min value: 0.0
	Alignment: 
	Mean: 0.708038687305	Standard deviation: 0.313022022896
	Max value: 1.0	Min value: 0.0
	Alignment e-utts: 
	Mean: 0.733221433739	Standard deviation: 0.355358167693
	Max value: 1.0	Min value: 0.0


In [4]:
print 'Differences between languages'
for var in results_all:
    mannwhitneyu =  stats.mannwhitneyu(results_de[var],results_en[var], alternative='two-sided')
    if mannwhitneyu[1] < 0.05:
        print var
        print str(mannwhitneyu)+'\tSignificantly different'
        print np.mean(results_de[var]), np.mean(results_en[var])
print '\n'

print 'Differences between players'
mannwhitneyu =  stats.mannwhitneyu(results_de['Alignment e-utts'],results_de['Alignment p-utts'], alternative='two-sided')
if mannwhitneyu[1] < 0.05:
    print 'Players move alignment de'
    print str(mannwhitneyu)+'\tSignificantly different'

print 'Differences between players'
mannwhitneyu =  stats.mannwhitneyu(results_en['Alignment e-utts'],results_en['Alignment p-utts'], alternative='two-sided')
if mannwhitneyu[1] < 0.05:
    print 'Players move alignment de'
    print str(mannwhitneyu)+'\tSignificantly different'
                                                                        
print 'Differences between players'
mannwhitneyu =  stats.mannwhitneyu(results_all['Alignment e-utts'],results_all['Alignment p-utts'], alternative='two-sided')
if mannwhitneyu[1] < 0.05:
    print 'Players move alignment de'
    print str(mannwhitneyu)+'\tSignificantly different'                                       
                                                                       

Differences between languages


Differences between players
Players move alignment de
MannwhitneyuResult(statistic=32475.5, pvalue=0.023703018853351143)	Significantly different
Differences between players
Players move alignment de
MannwhitneyuResult(statistic=7919.0, pvalue=0.026441435358971109)	Significantly different
Differences between players
Players move alignment de
MannwhitneyuResult(statistic=72333.0, pvalue=0.0021652700550864792)	Significantly different


In [16]:
### GAME LEVEL 
def compute_values_game(refexps_dir, success_dir):
    game_alignment = defaultdict(list)
    forms = []
    for dialogue in listdir(refexps_dir):
        dialogue = refexps.load_data(path.join(refexps_dir, dialogue))
        dialogue_game = refexps.gamerun_level(dialogue)
        for obj in dialogue_game:
            refobj = dialogue_game[obj]
            total_n = len(refobj)
            sameform_n = 0
            sameform_n_speaker = defaultdict(int)
            total_n_speaker = defaultdict(int)
            prev_forms = set()
            speakers = set()
            for ref in refobj:
                form = ref[3]
                forms.append(form)
                speaker = ref[1]
                if prev_forms != set():
                    if form in prev_forms:
                        sameform_n += 1
                        sameform_n_speaker[speaker] += 1
                else:
                    starter = speaker
                    speakers.add(speaker)
                total_n_speaker[speaker] += 1
                prev_forms.add(form)
            if (total_n - 1) != 0:
                alignment = float(sameform_n)/float(total_n - 1)
                game_alignment['Alignment'].append(alignment)
            for s in speakers:
                    if starter == s:
                        if (total_n_speaker[s] - 1) != 0:
                            alignment = float(sameform_n_speaker[s])/float(total_n_speaker[s] - 1)
                            game_alignment['Alignment '+s].append(alignment)
                    else:
                        if(total_n_speaker[s]) != 0:
                            alignment = float(sameform_n_speaker[s])/float(total_n_speaker[s])
                            game_alignment['Alignment '+s].append(alignment)
    for value in game_alignment:
        dataset = game_alignment[value]
        print '\t'+ value + ': '
        print '\tMean: '+ str(np.mean(dataset)) +'\tStandard deviation: '+ str(np.std(dataset))
        print '\tMax value: ' + str(max(dataset)) + '\tMin value: '+ str(min(dataset))
    print forms
    return game_alignment, forms

In [20]:
de_dataset_dir = 'En_De_Dataset/De/RefExp'
de_success_dir = 'En_De_Dataset/De/Success'

en_dataset_dir = 'En_De_Dataset/En/RefExp'
en_success_dir = 'En_De_Dataset/En/Success'

all_dataset_dir = 'En_De_Dataset/All/RefExp'
all_success_dir = 'En_De_Dataset/All/Success'

results_de, forms_de = compute_values_game(de_dataset_dir, de_success_dir)
print '\n\n'
results_en, forms_en = compute_values_game(en_dataset_dir, en_success_dir)
print '\n\n'
results_all, forms_all = compute_values_game(all_dataset_dir, all_success_dir)

	Alignment p-utts: 
	Mean: 0.702838328295	Standard deviation: 0.296473621952
	Max value: 1.0	Min value: 0.0
	Alignment: 
	Mean: 0.708780115712	Standard deviation: 0.288829910029
	Max value: 1.0	Min value: 0.0
	Alignment e-utts: 
	Mean: 0.652848639456	Standard deviation: 0.376833882275
	Max value: 1.0	Min value: 0.0
['pp', 'def_np', 'def_np', 'quant_np', 'def_np', 'def_np', 'quant_np', 'quant_np', 'quant_np', 'quant_np', 'pronoun', 'indef_np', 'indef_np', 'indef_np', 'indef_np', 'def_np', 'pronoun', 'pronoun', 'def_np', 'def_np', 'pronoun', 'def_np', 'pronoun', 'pronoun', 'def_np', 'def_np', 'def_np', 'def_np', 'pronoun', 'pronoun', 'def_np', 'pronoun', 'pronoun', 'pronoun', 'def_np', 'pronoun', 'pronoun', 'pronoun', 'def_np', 'pronoun', 'pronoun', 'pronoun', 'pp', 'attribution_sent', 'indef_np', 'pronoun', 'pronoun', 'def_np', 'pronoun', 'pronoun', 'pronoun', 'def_np', 'def_np', 'def_np', 'pronoun', 'pronoun', 'attribution_sent', 'def_np', 'indef_np', 'indef_np', 'indef_np', 'indef_np'

In [28]:
print 'Differences between languages'
for var in results_all:
    mannwhitneyu =  stats.mannwhitneyu(results_de[var],results_en[var], alternative='two-sided')
    if mannwhitneyu[1] < 0.05:
        print var
        print str(mannwhitneyu)+'\tSignificantly different'
        print np.mean(results_de[var]), np.mean(results_en[var])
print '\n'

cat_map = {key : i for key, i in zip(set(forms_de), range(len(set(forms_de))))}

print cat_map

enc_forms_de = [cat_map[f] for f in forms_de]
enc_forms_en = [cat_map[f] for f in forms_en]

from collections import Counter

print Counter(enc_forms_de)
print Counter(enc_forms_en)

print stats.mannwhitneyu(enc_forms_de, enc_forms_en, alternative='two-sided')
        
print 'Differences between players'
mannwhitneyu =  stats.mannwhitneyu(results_de['Alignment e-utts'],results_de['Alignment p-utts'], alternative='two-sided')
if mannwhitneyu[1] < 0.05:
    print 'Players move alignment de'
    print str(mannwhitneyu)+'\tSignificantly different'

mannwhitneyu =  stats.mannwhitneyu(results_en['Alignment e-utts'],results_en['Alignment p-utts'], alternative='two-sided')
if mannwhitneyu[1] < 0.05:
    print 'Players move alignment en'
    print str(mannwhitneyu)+'\tSignificantly different'
                                                                        
mannwhitneyu =  stats.mannwhitneyu(results_all['Alignment e-utts'],results_all['Alignment p-utts'], alternative='two-sided')
if mannwhitneyu[1] < 0.05:
    print 'Players move alignment all'
    print str(mannwhitneyu)+'\tSignificantly different'                                                             
                                                                    
    

Differences between languages


{'other_sent': 0, 'pp': 1, 'quant_np': 2, 'pronoun': 3, 'comp_sent': 4, 'indef_np': 5, 'other': 6, 'attribution_sent': 7, 'def_np': 8}
Counter({8: 1193, 3: 825, 5: 240, 1: 125, 2: 115, 4: 54, 7: 23, 0: 12, 6: 12})
Counter({8: 602, 3: 377, 5: 160, 2: 80, 1: 78, 4: 29, 7: 10, 0: 1, 6: 1})
MannwhitneyuResult(statistic=1752565.5, pvalue=0.66162013980093748)
Differences between players
